In [1]:
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms
from torchvision import datasets


In [2]:
train_dir = "/Users/malgosielska/Studies/Sem 1 TAI/Projekt wdrozeniowy/plantie-backend/plantvillage_dataset/train"
test_dir = "/Users/malgosielska/Studies/Sem 1 TAI/Projekt wdrozeniowy/plantie-backend/plantvillage_dataset/val"

train_dir, test_dir

('/Users/malgosielska/Studies/Sem 1 TAI/Projekt wdrozeniowy/plantie-backend/plantvillage_dataset/train',
 '/Users/malgosielska/Studies/Sem 1 TAI/Projekt wdrozeniowy/plantie-backend/plantvillage_dataset/val')

In [3]:
# Setup device-agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"

device

'cpu'

In [4]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
     transforms.Resize(size=(28, 28)),
]) # maybe we need to resize??

In [5]:
train_data = datasets.ImageFolder(root=train_dir, 
                                  transform=transform)

test_data = datasets.ImageFolder(root=test_dir, 
                                  transform=transform)

train_data, test_data


(Dataset ImageFolder
     Number of datapoints: 43444
     Root location: /Users/malgosielska/Studies/Sem 1 TAI/Projekt wdrozeniowy/plantie-backend/plantvillage_dataset/train
     StandardTransform
 Transform: Compose(
                ToTensor()
                Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
                Resize(size=(28, 28), interpolation=bilinear, max_size=None, antialias=True)
            ),
 Dataset ImageFolder
     Number of datapoints: 10861
     Root location: /Users/malgosielska/Studies/Sem 1 TAI/Projekt wdrozeniowy/plantie-backend/plantvillage_dataset/val
     StandardTransform
 Transform: Compose(
                ToTensor()
                Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
                Resize(size=(28, 28), interpolation=bilinear, max_size=None, antialias=True)
            ))

In [6]:
class_names = train_data.classes
class_dict = train_data.class_to_idx
class_dict, len(class_names)

({'Apple___Apple_scab': 0,
  'Apple___Black_rot': 1,
  'Apple___Cedar_apple_rust': 2,
  'Apple___healthy': 3,
  'Blueberry___healthy': 4,
  'Cherry_(including_sour)___Powdery_mildew': 5,
  'Cherry_(including_sour)___healthy': 6,
  'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot': 7,
  'Corn_(maize)___Common_rust_': 8,
  'Corn_(maize)___Northern_Leaf_Blight': 9,
  'Corn_(maize)___healthy': 10,
  'Grape___Black_rot': 11,
  'Grape___Esca_(Black_Measles)': 12,
  'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)': 13,
  'Grape___healthy': 14,
  'Orange___Haunglongbing_(Citrus_greening)': 15,
  'Peach___Bacterial_spot': 16,
  'Peach___healthy': 17,
  'Pepper,_bell___Bacterial_spot': 18,
  'Pepper,_bell___healthy': 19,
  'Potato___Early_blight': 20,
  'Potato___Late_blight': 21,
  'Potato___healthy': 22,
  'Raspberry___healthy': 23,
  'Soybean___healthy': 24,
  'Squash___Powdery_mildew': 25,
  'Strawberry___Leaf_scorch': 26,
  'Strawberry___healthy': 27,
  'Tomato___Bacterial_spot': 28,
  'Toma

In [7]:
import os 

BATCH_SIZE = 32
NUM_WORKERS = os.cpu_count()

train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=True, num_workers=2)

train_loader, test_loader, NUM_WORKERS

(<torch.utils.data.dataloader.DataLoader at 0x13c0d1b80>,
 10)

In [8]:
image, label = train_data[0]

image, label, image.size()

(tensor([[[ 2.0596e-02,  1.1013e-02,  3.6671e-03,  ..., -3.8001e-02,
           -2.7272e-02, -2.9280e-02],
          [ 1.7149e-02,  1.4004e-02,  3.7120e-03,  ..., -3.5130e-02,
           -2.8804e-02, -1.5381e-02],
          [ 1.5961e-02,  1.3483e-02,  1.1774e-04,  ..., -4.7084e-02,
           -3.4970e-02, -1.7318e-02],
          ...,
          [-3.9131e-01, -5.1561e-01, -6.7428e-01,  ..., -3.0862e-01,
           -2.8105e-01, -2.5933e-01],
          [-4.4858e-01, -4.9426e-01, -6.2021e-01,  ..., -3.1413e-01,
           -2.9239e-01, -2.8838e-01],
          [-5.8006e-01, -5.7658e-01, -6.1283e-01,  ..., -3.2462e-01,
           -3.1566e-01, -3.1754e-01]],
 
         [[-3.4306e-02, -3.7273e-02, -2.8429e-02,  ..., -6.2032e-02,
           -6.2077e-02, -6.8495e-02],
          [-3.7753e-02, -3.4282e-02, -2.8385e-02,  ..., -6.3450e-02,
           -6.4849e-02, -5.4596e-02],
          [-3.8941e-02, -3.4803e-02, -3.1979e-02,  ..., -8.5833e-02,
           -7.4050e-02, -5.6533e-02],
          ...,
    

In [9]:
class NeuralNet(nn.Module):

    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(3, 12, 5) # (28 - 5) / 1 + 1 = 24
        # (12, 24, 24)
        self.pool = nn.MaxPool2d(2,2)
        # (12, 12, 12)
        self.conv2 = nn.Conv2d(12, 24, 5) # (12 - 5) / 1 + 1 = 8
        # (24, 8, 8) -> (24, 4, 4) -> Flatten(24 * 4 * 4)

        self.fc1 = nn.Linear(24 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 38) # 38 is the number of classes, make it a parameter

    def forward(self,  x: torch.Tensor):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return x




In [10]:
net = NeuralNet()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [11]:
for epoch in range(5):
    print(f"Epoch {epoch + 1} / 5")
    running_loss = 0.0

    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = net(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f"Loss: {running_loss / len(train_loader):.4f}")

Epoch 1 / 5
Loss: 3.5851
Epoch 2 / 5
Loss: 3.4349
Epoch 3 / 5
Loss: 3.1765
Epoch 4 / 5
Loss: 2.6948
Epoch 5 / 5
Loss: 2.0765


In [14]:
torch.save(net.state_dict(), "plant_disease_model.pth")

In [26]:
net = NeuralNet()
net.load_state_dict(torch.load("plant_disease_model.pth"))
net.to(device)

NeuralNet(
  (conv1): Conv2d(3, 12, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 24, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=384, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=38, bias=True)
)

In [ ]:
correct = 0
total = 0 

net.eval()
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)

        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy of the network on the test images: {accuracy:.2f}%")


Accuracy of the network on the test images: 51.88%


['Apple___Apple_scab',
 'Apple___Black_rot',
 'Apple___Cedar_apple_rust',
 'Apple___healthy',
 'Blueberry___healthy',
 'Cherry_(including_sour)___Powdery_mildew',
 'Cherry_(including_sour)___healthy',
 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
 'Corn_(maize)___Common_rust_',
 'Corn_(maize)___Northern_Leaf_Blight',
 'Corn_(maize)___healthy',
 'Grape___Black_rot',
 'Grape___Esca_(Black_Measles)',
 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
 'Grape___healthy',
 'Orange___Haunglongbing_(Citrus_greening)',
 'Peach___Bacterial_spot',
 'Peach___healthy',
 'Pepper,_bell___Bacterial_spot',
 'Pepper,_bell___healthy',
 'Potato___Early_blight',
 'Potato___Late_blight',
 'Potato___healthy',
 'Raspberry___healthy',
 'Soybean___healthy',
 'Squash___Powdery_mildew',
 'Strawberry___Leaf_scorch',
 'Strawberry___healthy',
 'Tomato___Bacterial_spot',
 'Tomato___Early_blight',
 'Tomato___Late_blight',
 'Tomato___Leaf_Mold',
 'Tomato___Septoria_leaf_spot',
 'Tomato___Spider_mites Two-spotted_

In [30]:
new_transform = transforms.Compose([
    transforms.Resize((28, 28)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

def load_image(image_path):
    image = Image.open(image_path)
    image = new_transform(image)
    image = image.unsqueeze(0)  # Add batch dimension
    return image

image_paths = ["0a14783a-838a-4d4f-a671-ff98011714c6___FREC_Scab 3288.JPG", "0b8dabb7-5f1b-4fdc-b3fa-30b289707b90___JR_FrgE.S 3047.JPG"]
images = [load_image(path) for path in image_paths]

net.eval()
with torch.no_grad():
    for image in images:
        output = net(image)
        _, predicted = torch.max(output, 1)
        print(f"Predicted class: {class_names[predicted.item()]}")

Predicted class: Tomato___Early_blight
Predicted class: Apple___Black_rot
